In [34]:
import pandas as pd

df = pd.read_csv("../final_data/data_260116.csv")

pd.set_option("display.max_columns", None)
df.head()

,experiment_date,treatment,source,ops_type_merged,city_group,user_cnt,nonrepeat_cnt,trip_cnt,weekday_nonrepeat_cnt,weekend_nonrepeat_cnt,weekday_trip_cnt,weekend_trip_cnt,nonrepeat_cnt_per_user,trip_cnt_per_user,weekday_nonrepeat_cnt_per_user,weekend_nonrepeat_cnt_per_user,weekday_trip_cnt_per_user,weekend_trip_cnt_per_user,weekday_match_rate,weekend_match_rate,coupon_BD_total,coupon_CDP_total,coupon_folk_total,coupon_growth_other_total,coupon_MGM_total,coupon_MKT_total,coupon_register_total,coupon_daily_total,avg_rainy_day,avg_rainy_weekday,avg_rainy_weekend,mgm_day
0,2025-07-28,不發,控制組,14天在其他尖峰預估車資,中區,333,91,45,74,17,34,11,0.273273,0.135135,0.222222,0.051051,0.102102,0.033033,0.523529,0.749091,20,114,0,2,9,0,24,0,0.897959,0.000000,0.500000,7
1,2025-07-28,不發,控制組,14天在其他尖峰預估車資,北區,1220,228,161,165,63,129,32,0.186885,0.131967,0.135246,0.051639,0.105738,0.026230,0.793101,0.779062,107,583,0,12,299,0,30,1,0.732143,0.000000,0.000000,7
2,2025-07-28,不發,控制組,14天在其他尖峰預估車資,南區,374,73,51,52,21,36,15,0.195187,0.136364,0.139037,0.056150,0.096257,0.040107,0.751944,0.850000,30,129,1,2,10,1,21,0,0.809524,0.366667,0.583333,7
3,2025-07-28,不發,控制組,14天在晚尖峰預估車資,中區,106,34,23,23,11,15,8,0.320755,0.216981,0.216981,0.103774,0.141509,0.075472,0.744000,0.917500,10,36,0,2,1,0,0,0,0.897959,0.000000,0.500000,7
4,2025-07-28,不發,控制組,14天在晚尖峰預估車資,北區,288,56,42,48,8,37,5,0.194444,0.145833,0.166667,0.027778,0.128472,0.017361,0.845676,0.750000,37,90,0,2,56,0,6,0,0.732143,0.000000,0.000000,7


In [35]:
import pandas as pd
import numpy as np

IN_PATH = "../final_data/data_260116.csv"

# ----------------------------
# Config
# ----------------------------
GROUP_COLS = ["treatment", "source", "ops_type_merged", "city_group"]
# Try common week order columns; fall back to experiment_date if present
CANDIDATE_TIME_COLS = ["experiment_date", "week", "week_start", "week_begin", "date", "Dates"]

TARGET_COLS = [
    "nonrepeat_cnt_per_user",
    "trip_cnt_per_user",
    "weekday_nonrepeat_cnt_per_user",
    "weekend_nonrepeat_cnt_per_user",
    "weekday_trip_cnt_per_user",
    "weekend_trip_cnt_per_user",
    "weekday_match_rate",
    "weekend_match_rate",
    "coupon_BD_total",
    "coupon_CDP_total",
    "coupon_folk_total",
    "coupon_growth_other_total",
    "coupon_MGM_total",
    "coupon_MKT_total",
    "coupon_register_total",
    "coupon_daily_total",
]

# ----------------------------
# Load
# ----------------------------
df = pd.read_csv(IN_PATH)

# ----------------------------
# Pick time column and parse
# ----------------------------
time_col = None
for c in CANDIDATE_TIME_COLS:
    if c in df.columns:
        time_col = c
        break

if time_col is None:
    raise ValueError(
        "找不到週次/時間欄位。請確認資料是否有 experiment_date 或 week 等欄位。"
    )

# If experiment_date-like, parse to datetime for correct sorting
if df[time_col].dtype == "object":
    # Try parse; if fails, keep original
    parsed = pd.to_datetime(df[time_col], errors="coerce")
    # Use parsed when it parsed enough values
    if parsed.notna().mean() > 0.5:
        df[time_col] = parsed

# ----------------------------
# Ensure required columns exist
# ----------------------------
missing_group = [c for c in GROUP_COLS if c not in df.columns]
missing_target = [c for c in TARGET_COLS if c not in df.columns]

if missing_group:
    raise ValueError(f"缺少分組欄位: {missing_group}")
if missing_target:
    raise ValueError(f"缺少目標欄位: {missing_target}")

# Make sure targets are numeric (keep NaN if non-numeric)
for c in TARGET_COLS:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ----------------------------
# Sort and compute lags / roll4
# Rules per your request:
# - lag1: 1 week before (previous row in that group's time order), else NaN
# - lag2: 2 weeks before, else NaN
# - roll4: mean of previous 4 weeks ONLY if all 4 exist, else NaN (min_periods=4)
# ----------------------------
df = df.sort_values(GROUP_COLS + [time_col]).reset_index(drop=True)

g = df.groupby(GROUP_COLS, sort=False)

for c in TARGET_COLS:
    df[f"{c}_lag1"] = g[c].shift(1)
    df[f"{c}_lag2"] = g[c].shift(2)
    df[f"{c}_roll4"] = g[c].shift(1).rolling(window=4, min_periods=4).mean()

# ----------------------------
# Show a quick check
# ----------------------------
print("time_col used for ordering:", time_col)
print("shape:", df.shape)
df.head()

time_col used for ordering: experiment_date
shape: (3072, 80)


,experiment_date,treatment,source,ops_type_merged,city_group,user_cnt,nonrepeat_cnt,trip_cnt,weekday_nonrepeat_cnt,weekend_nonrepeat_cnt,weekday_trip_cnt,weekend_trip_cnt,nonrepeat_cnt_per_user,trip_cnt_per_user,weekday_nonrepeat_cnt_per_user,weekend_nonrepeat_cnt_per_user,weekday_trip_cnt_per_user,weekend_trip_cnt_per_user,weekday_match_rate,weekend_match_rate,coupon_BD_total,coupon_CDP_total,coupon_folk_total,coupon_growth_other_total,coupon_MGM_total,coupon_MKT_total,coupon_register_total,coupon_daily_total,avg_rainy_day,avg_rainy_weekday,avg_rainy_weekend,mgm_day,nonrepeat_cnt_per_user_lag1,nonrepeat_cnt_per_user_lag2,nonrepeat_cnt_per_user_roll4,trip_cnt_per_user_lag1,trip_cnt_per_user_lag2,trip_cnt_per_user_roll4,weekday_nonrepeat_cnt_per_user_lag1,weekday_nonrepeat_cnt_per_user_lag2,weekday_nonrepeat_cnt_per_user_roll4,weekend_nonrepeat_cnt_per_user_lag1,weekend_nonrepeat_cnt_per_user_lag2,weekend_nonrepeat_cnt_per_user_roll4,weekday_trip_cnt_per_user_lag1,weekday_trip_cnt_per_user_lag2,weekday_trip_cnt_per_user_roll4,weekend_trip_cnt_per_user_lag1,weekend_trip_cnt_per_user_lag2,weekend_trip_cnt_per_user_roll4,weekday_match_rate_lag1,weekday_match_rate_lag2,weekday_match_rate_roll4,weekend_match_rate_lag1,weekend_match_rate_lag2,weekend_match_rate_roll4,coupon_BD_total_lag1,coupon_BD_total_lag2,coupon_BD_total_roll4,coupon_CDP_total_lag1,coupon_CDP_total_lag2,coupon_CDP_total_roll4,coupon_folk_total_lag1,coupon_folk_total_lag2,coupon_folk_total_roll4,coupon_growth_other_total_lag1,coupon_growth_other_total_lag2,coupon_growth_other_total_roll4,coupon_MGM_total_lag1,coupon_MGM_total_lag2,coupon_MGM_total_roll4,coupon_MKT_total_lag1,coupon_MKT_total_lag2,coupon_MKT_total_roll4,coupon_register_total_lag1,coupon_register_total_lag2,coupon_register_total_roll4,coupon_daily_total_lag1,coupon_daily_total_lag2,coupon_daily_total_roll4
0,2025-11-03,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,193,53,42,41,12,38,4,0.274611,0.217617,0.212435,0.062176,0.196891,0.020725,0.853684,0.752500,82,124,386,6,0,25,6,0,0.000000,0.000000,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-11-10,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,71,15,11,12,3,6,5,0.211268,0.154930,0.169014,0.042254,0.084507,0.070423,0.916667,0.830000,24,29,142,0,0,0,6,0,0.244898,0.228571,0.0,0,0.274611,NaN,NaN,0.217617,NaN,NaN,0.212435,NaN,NaN,0.062176,NaN,NaN,0.196891,NaN,NaN,0.020725,NaN,NaN,0.853684,NaN,NaN,0.752500,NaN,NaN,82.0,NaN,NaN,124.0,NaN,NaN,386.0,NaN,NaN,6.0,NaN,NaN,0.0,NaN,NaN,25.0,NaN,NaN,6.0,NaN,NaN,0.0,NaN,NaN
2,2025-11-17,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,189,46,28,36,10,20,8,0.243386,0.148148,0.190476,0.052910,0.105820,0.042328,0.821500,0.650000,82,127,378,4,0,18,8,0,0.000000,0.000000,0.0,0,0.211268,0.274611,NaN,0.154930,0.217617,NaN,0.169014,0.212435,NaN,0.042254,0.062176,NaN,0.084507,0.196891,NaN,0.070423,0.020725,NaN,0.916667,0.853684,NaN,0.830000,0.7525,NaN,24.0,82.0,NaN,29.0,124.0,NaN,142.0,386.0,NaN,0.0,6.0,NaN,0.0,0.0,NaN,0.0,25.0,NaN,6.0,6.0,NaN,0.0,0.0,NaN
3,2025-11-24,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,184,38,29,31,7,22,7,0.206522,0.157609,0.168478,0.038043,0.119565,0.038043,0.835909,0.728571,69,68,368,0,0,15,3,0,0.000000,0.000000,0.0,5,0.243386,0.211268,NaN,0.148148,0.154930,NaN,0.190476,0.169014,NaN,0.052910,0.042254,NaN,0.105820,0.084507,NaN,0.042328,0.070423,NaN,0.821500,0.916667,NaN,0.650000,0.8300,NaN,82.0,24.0,NaN,127.0,29.0,NaN,378.0,142.0,NaN,4.0,0.0,NaN,0.0,0.0,NaN,18.0,0.0,NaN,8.0,6.0,NaN,0.0,0.0,NaN
4,2025-12-01,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,185,33,24,19,14,14,10,0.178378,0.129730,0.102703,0.075676,0.075676,0.054054,0.904286,0.768000,76,101,370,0,3,20,4,0,0.000000,0.000000,0.0,2,0.206522,0.243386,0.233947,0.157609,0.148148,0.169576,0.168478,0.190476,0.185101,0.038043,0.052910,0.048846,0.119565,0.105820,0.126696,0.038043,0.042328,0.04288,0.835909,0.821500,0.85694,0.728571,0.6500,0.740268,69.0,82.0,64.25,68.0,127.0,87.0,368.0,3

In [36]:
import numpy as np

# 1) 差 (lag1 - lag2)
df["delta_trip_per_user"] = df["trip_cnt_per_user_lag1"] - df["trip_cnt_per_user_lag2"]
df["delta_nonrepeat_per_user"] = df["nonrepeat_cnt_per_user_lag1"] - df["nonrepeat_cnt_per_user_lag2"]

# 2) 比例 (lag1 / lag2)；lag2=0 -> NaN（避免 inf）
df["ratio_trip_per_user"] = df["trip_cnt_per_user_lag1"] / df["trip_cnt_per_user_lag2"].replace(0, np.nan)
df["ratio_nonrepeat_per_user"] = df["nonrepeat_cnt_per_user_lag1"] / df["nonrepeat_cnt_per_user_lag2"].replace(0, np.nan)

df.head()

,experiment_date,treatment,source,ops_type_merged,city_group,user_cnt,nonrepeat_cnt,trip_cnt,weekday_nonrepeat_cnt,weekend_nonrepeat_cnt,weekday_trip_cnt,weekend_trip_cnt,nonrepeat_cnt_per_user,trip_cnt_per_user,weekday_nonrepeat_cnt_per_user,weekend_nonrepeat_cnt_per_user,weekday_trip_cnt_per_user,weekend_trip_cnt_per_user,weekday_match_rate,weekend_match_rate,coupon_BD_total,coupon_CDP_total,coupon_folk_total,coupon_growth_other_total,coupon_MGM_total,coupon_MKT_total,coupon_register_total,coupon_daily_total,avg_rainy_day,avg_rainy_weekday,avg_rainy_weekend,mgm_day,nonrepeat_cnt_per_user_lag1,nonrepeat_cnt_per_user_lag2,nonrepeat_cnt_per_user_roll4,trip_cnt_per_user_lag1,trip_cnt_per_user_lag2,trip_cnt_per_user_roll4,weekday_nonrepeat_cnt_per_user_lag1,weekday_nonrepeat_cnt_per_user_lag2,weekday_nonrepeat_cnt_per_user_roll4,weekend_nonrepeat_cnt_per_user_lag1,weekend_nonrepeat_cnt_per_user_lag2,weekend_nonrepeat_cnt_per_user_roll4,weekday_trip_cnt_per_user_lag1,weekday_trip_cnt_per_user_lag2,weekday_trip_cnt_per_user_roll4,weekend_trip_cnt_per_user_lag1,weekend_trip_cnt_per_user_lag2,weekend_trip_cnt_per_user_roll4,weekday_match_rate_lag1,weekday_match_rate_lag2,weekday_match_rate_roll4,weekend_match_rate_lag1,weekend_match_rate_lag2,weekend_match_rate_roll4,coupon_BD_total_lag1,coupon_BD_total_lag2,coupon_BD_total_roll4,coupon_CDP_total_lag1,coupon_CDP_total_lag2,coupon_CDP_total_roll4,coupon_folk_total_lag1,coupon_folk_total_lag2,coupon_folk_total_roll4,coupon_growth_other_total_lag1,coupon_growth_other_total_lag2,coupon_growth_other_total_roll4,coupon_MGM_total_lag1,coupon_MGM_total_lag2,coupon_MGM_total_roll4,coupon_MKT_total_lag1,coupon_MKT_total_lag2,coupon_MKT_total_roll4,coupon_register_total_lag1,coupon_register_total_lag2,coupon_register_total_roll4,coupon_daily_total_lag1,coupon_daily_total_lag2,coupon_daily_total_roll4,delta_trip_per_user,delta_nonrepeat_per_user,ratio_trip_per_user,ratio_nonrepeat_per_user
0,2025-11-03,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,193,53,42,41,12,38,4,0.274611,0.217617,0.212435,0.062176,0.196891,0.020725,0.853684,0.752500,82,124,386,6,0,25,6,0,0.000000,0.000000,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-11-10,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,71,15,11,12,3,6,5,0.211268,0.154930,0.169014,0.042254,0.084507,0.070423,0.916667,0.830000,24,29,142,0,0,0,6,0,0.244898,0.228571,0.0,0,0.274611,NaN,NaN,0.217617,NaN,NaN,0.212435,NaN,NaN,0.062176,NaN,NaN,0.196891,NaN,NaN,0.020725,NaN,NaN,0.853684,NaN,NaN,0.752500,NaN,NaN,82.0,NaN,NaN,124.0,NaN,NaN,386.0,NaN,NaN,6.0,NaN,NaN,0.0,NaN,NaN,25.0,NaN,NaN,6.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-11-17,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,189,46,28,36,10,20,8,0.243386,0.148148,0.190476,0.052910,0.105820,0.042328,0.821500,0.650000,82,127,378,4,0,18,8,0,0.000000,0.000000,0.0,0,0.211268,0.274611,NaN,0.154930,0.217617,NaN,0.169014,0.212435,NaN,0.042254,0.062176,NaN,0.084507,0.196891,NaN,0.070423,0.020725,NaN,0.916667,0.853684,NaN,0.830000,0.7525,NaN,24.0,82.0,NaN,29.0,124.0,NaN,142.0,386.0,NaN,0.0,6.0,NaN,0.0,0.0,NaN,0.0,25.0,NaN,6.0,6.0,NaN,0.0,0.0,NaN,-0.062687,-0.063344,0.711938,0.769333
3,2025-11-24,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,184,38,29,31,7,22,7,0.206522,0.157609,0.168478,0.038043,0.119565,0.038043,0.835909,0.728571,69,68,368,0,0,15,3,0,0.000000,0.000000,0.0,5,0.243386,0.211268,NaN,0.148148,0.154930,NaN,0.190476,0.169014,NaN,0.052910,0.042254,NaN,0.105820,0.084507,NaN,0.042328,0.070423,NaN,0.821500,0.916667,NaN,0.650000,0.8300,NaN,82.0,24.0,NaN,127.0,29.0,NaN,378.0,142.0,NaN,4.0,0.0,NaN,0.0,0.0,NaN,18.0,0.0,NaN,8.0,6.0,NaN,0.0,0.0,NaN,-0.006781,0.032119,0.956229,1.152028
4,2025-12-01,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,185,33,24,19,14,14,10,0.178378,0.129730,0.102703,0.075676,0.075676,0.054054,0.904286,0.768000,76,101,370,0,3,20,4,0,0.000000,0.000000,0.0,2,0.206522,0.243386,0.233947,0.157609,0.148148,0.169576,

In [32]:
import pandas as pd
import re
import numpy as np

def parse_face_value(t: str):
    if pd.isna(t):
        return (np.nan, np.nan)

    t = str(t).strip()

    if t == "不發":
        return (0, 0)

    # 允許：
    # - "15元1張"          -> (15, 1)
    # - "15x2元1張"        -> (15, 2)
    m = re.fullmatch(r"(\d+)(?:x(\d+))?元1張", t)
    if not m:
        # 若遇到不在清單內的值，直接回 NaN，避免亂猜
        return (np.nan, np.nan)

    face = int(m.group(1))
    num = int(m.group(2)) if m.group(2) is not None else 1
    return (face, num)

# 解析成兩欄
parsed = df["treatment"].apply(parse_face_value)
df["face_value"] = parsed.apply(lambda x: x[0])
df["face_value_num"] = parsed.apply(lambda x: x[1])

# （可選）檢查是否有解析失敗的 treatment
bad = df.loc[df["face_value"].isna() | df["face_value_num"].isna(), "treatment"].dropna().unique()
print("Unparsed treatment values:", bad[:20], " ... count=", len(bad))

# ===== 新增：把指定 coupon_*_total_* 欄位 / user_cnt，並移除 total 欄位 =====
coupon_total_cols = [
    "coupon_BD_total_lag1", "coupon_BD_total_lag2", "coupon_BD_total_roll4",
    "coupon_CDP_total_lag1", "coupon_CDP_total_lag2", "coupon_CDP_total_roll4",
    "coupon_folk_total_lag1", "coupon_folk_total_lag2", "coupon_folk_total_roll4",
    "coupon_growth_other_total_lag1", "coupon_growth_other_total_lag2", "coupon_growth_other_total_roll4",
    "coupon_MGM_total_lag1", "coupon_MGM_total_lag2", "coupon_MGM_total_roll4",
    "coupon_MKT_total_lag1", "coupon_MKT_total_lag2", "coupon_MKT_total_roll4",
    "coupon_register_total_lag1", "coupon_register_total_lag2", "coupon_register_total_roll4",
    "coupon_daily_total_lag1", "coupon_daily_total_lag2", "coupon_daily_total_roll4",
]

# 只處理 df 裡 реально 存在的欄位（避免 KeyError）
coupon_total_cols = [c for c in coupon_total_cols if c in df.columns]

den = pd.to_numeric(df["user_cnt"], errors="coerce").replace(0, np.nan)

for c in coupon_total_cols:
    per_user_col = c.replace("_total_", "_per_user_")  # e.g. coupon_BD_total_lag1 -> coupon_BD_per_user_lag1
    df[per_user_col] = pd.to_numeric(df[c], errors="coerce") / den

# 不保留 total 欄位
df.drop(columns=coupon_total_cols, inplace=True)

df.head()


Unparsed treatment values: []  ... count= 0


,experiment_date,treatment,source,ops_type_merged,city_group,user_cnt,nonrepeat_cnt,trip_cnt,weekday_nonrepeat_cnt,weekend_nonrepeat_cnt,weekday_trip_cnt,weekend_trip_cnt,nonrepeat_cnt_per_user,trip_cnt_per_user,weekday_nonrepeat_cnt_per_user,weekend_nonrepeat_cnt_per_user,weekday_trip_cnt_per_user,weekend_trip_cnt_per_user,weekday_match_rate,weekend_match_rate,coupon_BD_total,coupon_CDP_total,coupon_folk_total,coupon_growth_other_total,coupon_MGM_total,coupon_MKT_total,coupon_register_total,coupon_daily_total,avg_rainy_day,avg_rainy_weekday,avg_rainy_weekend,mgm_day,nonrepeat_cnt_per_user_lag1,nonrepeat_cnt_per_user_lag2,nonrepeat_cnt_per_user_roll4,trip_cnt_per_user_lag1,trip_cnt_per_user_lag2,trip_cnt_per_user_roll4,weekday_nonrepeat_cnt_per_user_lag1,weekday_nonrepeat_cnt_per_user_lag2,weekday_nonrepeat_cnt_per_user_roll4,weekend_nonrepeat_cnt_per_user_lag1,weekend_nonrepeat_cnt_per_user_lag2,weekend_nonrepeat_cnt_per_user_roll4,weekday_trip_cnt_per_user_lag1,weekday_trip_cnt_per_user_lag2,weekday_trip_cnt_per_user_roll4,weekend_trip_cnt_per_user_lag1,weekend_trip_cnt_per_user_lag2,weekend_trip_cnt_per_user_roll4,weekday_match_rate_lag1,weekday_match_rate_lag2,weekday_match_rate_roll4,weekend_match_rate_lag1,weekend_match_rate_lag2,weekend_match_rate_roll4,delta_trip_per_user,delta_nonrepeat_per_user,ratio_trip_per_user,ratio_nonrepeat_per_user,face_value,face_value_num,coupon_BD_per_user_lag1,coupon_BD_per_user_lag2,coupon_BD_per_user_roll4,coupon_CDP_per_user_lag1,coupon_CDP_per_user_lag2,coupon_CDP_per_user_roll4,coupon_folk_per_user_lag1,coupon_folk_per_user_lag2,coupon_folk_per_user_roll4,coupon_growth_other_per_user_lag1,coupon_growth_other_per_user_lag2,coupon_growth_other_per_user_roll4,coupon_MGM_per_user_lag1,coupon_MGM_per_user_lag2,coupon_MGM_per_user_roll4,coupon_MKT_per_user_lag1,coupon_MKT_per_user_lag2,coupon_MKT_per_user_roll4,coupon_register_per_user_lag1,coupon_register_per_user_lag2,coupon_register_per_user_roll4,coupon_daily_per_user_lag1,coupon_daily_per_user_lag2,coupon_daily_per_user_roll4
0,2025-11-03,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,193,53,42,41,12,38,4,0.274611,0.217617,0.212435,0.062176,0.196891,0.020725,0.853684,0.752500,82,124,386,6,0,25,6,0,0.000000,0.000000,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-11-10,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,71,15,11,12,3,6,5,0.211268,0.154930,0.169014,0.042254,0.084507,0.070423,0.916667,0.830000,24,29,142,0,0,0,6,0,0.244898,0.228571,0.0,0,0.274611,NaN,NaN,0.217617,NaN,NaN,0.212435,NaN,NaN,0.062176,NaN,NaN,0.196891,NaN,NaN,0.020725,NaN,NaN,0.853684,NaN,NaN,0.752500,NaN,NaN,NaN,NaN,NaN,NaN,15,2,1.154930,NaN,NaN,1.746479,NaN,NaN,5.436620,NaN,NaN,0.084507,NaN,NaN,0.0,NaN,NaN,0.352113,NaN,NaN,0.084507,NaN,NaN,0.0,NaN,NaN
2,2025-11-17,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,189,46,28,36,10,20,8,0.243386,0.148148,0.190476,0.052910,0.105820,0.042328,0.821500,0.650000,82,127,378,4,0,18,8,0,0.000000,0.000000,0.0,0,0.211268,0.274611,NaN,0.154930,0.217617,NaN,0.169014,0.212435,NaN,0.042254,0.062176,NaN,0.084507,0.196891,NaN,0.070423,0.020725,NaN,0.916667,0.853684,NaN,0.830000,0.7525,NaN,-0.062687,-0.063344,0.711938,0.769333,15,2,0.126984,0.433862,NaN,0.153439,0.656085,NaN,0.751323,2.042328,NaN,0.000000,0.031746,NaN,0.0,0.0,NaN,0.000000,0.132275,NaN,0.031746,0.031746,NaN,0.0,0.0,NaN
3,2025-11-24,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,184,38,29,31,7,22,7,0.206522,0.157609,0.168478,0.038043,0.119565,0.038043,0.835909,0.728571,69,68,368,0,0,15,3,0,0.000000,0.000000,0.0,5,0.243386,0.211268,NaN,0.148148,0.154930,NaN,0.190476,0.169014,NaN,0.052910,0.042254,NaN,0.105820,0.084507,NaN,0.042328,0.070423,NaN,0.821500,0.916667,NaN,0.650000,0.8300,NaN,-0.006781,0.032119,0.956229,1.152028,15,2,0.445652,0.130435,NaN,0.690217,0.157609,NaN,2.054348,0.771739,NaN,0.021739,0.000000,NaN,0.0,0.0,NaN,0.097826,0.000000,NaN,0.043478,0.032609,NaN,0.0,0

In [37]:
import re
import numpy as np
import pandas as pd

def parse_face_value(t: str):
    if pd.isna(t):
        return (np.nan, np.nan)

    t = str(t).strip()

    if t == "不發":
        return (0, 0)

    m = re.fullmatch(r"(\d+)(?:x(\d+))?元1張", t)
    if not m:
        return (np.nan, np.nan)

    face = int(m.group(1))
    num = int(m.group(2)) if m.group(2) is not None else 1
    return (face, num)

# -------------------------
# 1) treatment -> face_value, face_value_num
# -------------------------
parsed = df["treatment"].apply(parse_face_value)
df["face_value"] = parsed.apply(lambda x: x[0])
df["face_value_num"] = parsed.apply(lambda x: x[1])

bad = df.loc[df["face_value"].isna() | df["face_value_num"].isna(), "treatment"].dropna().unique()
print("Unparsed treatment values:", bad[:20], " ... count=", len(bad))

# -------------------------
# 2) coupon 欄位新增 log1p 欄位
#    命名規則：
#      - 若欄名含 _lag1 / _lag2 / _roll4：
#           coupon_xxx_lag1  -> coupon_xxx_log1p_lag1
#      - 若不含上述 suffix：
#           coupon_xxx       -> coupon_xxx_log1p
# -------------------------
coupon_cols = [c for c in df.columns if "coupon" in c.lower()]

suffixes = ("_lag1", "_lag2", "_roll4")

def make_log1p_name(col: str) -> str:
    for suf in suffixes:
        if col.endswith(suf):
            return col[:-len(suf)] + "_log1p" + suf
    return col + "_log1p"

created = []
for c in coupon_cols:
    s = pd.to_numeric(df[c], errors="coerce")
    new_c = make_log1p_name(c)
    df[new_c] = np.log1p(s)
    created.append(new_c)

print(f"Created {len(created)} log1p columns.")
print("Example new columns:", created[:10])

df.head()


Unparsed treatment values: []  ... count= 0
Created 32 log1p columns.
Example new columns: ['coupon_BD_total_log1p', 'coupon_CDP_total_log1p', 'coupon_folk_total_log1p', 'coupon_growth_other_total_log1p', 'coupon_MGM_total_log1p', 'coupon_MKT_total_log1p', 'coupon_register_total_log1p', 'coupon_daily_total_log1p', 'coupon_BD_total_log1p_lag1', 'coupon_BD_total_log1p_lag2']


,experiment_date,treatment,source,ops_type_merged,city_group,user_cnt,nonrepeat_cnt,trip_cnt,weekday_nonrepeat_cnt,weekend_nonrepeat_cnt,weekday_trip_cnt,weekend_trip_cnt,nonrepeat_cnt_per_user,trip_cnt_per_user,weekday_nonrepeat_cnt_per_user,weekend_nonrepeat_cnt_per_user,weekday_trip_cnt_per_user,weekend_trip_cnt_per_user,weekday_match_rate,weekend_match_rate,coupon_BD_total,coupon_CDP_total,coupon_folk_total,coupon_growth_other_total,coupon_MGM_total,coupon_MKT_total,coupon_register_total,coupon_daily_total,avg_rainy_day,avg_rainy_weekday,avg_rainy_weekend,mgm_day,nonrepeat_cnt_per_user_lag1,nonrepeat_cnt_per_user_lag2,nonrepeat_cnt_per_user_roll4,trip_cnt_per_user_lag1,trip_cnt_per_user_lag2,trip_cnt_per_user_roll4,weekday_nonrepeat_cnt_per_user_lag1,weekday_nonrepeat_cnt_per_user_lag2,weekday_nonrepeat_cnt_per_user_roll4,weekend_nonrepeat_cnt_per_user_lag1,weekend_nonrepeat_cnt_per_user_lag2,weekend_nonrepeat_cnt_per_user_roll4,weekday_trip_cnt_per_user_lag1,weekday_trip_cnt_per_user_lag2,weekday_trip_cnt_per_user_roll4,weekend_trip_cnt_per_user_lag1,weekend_trip_cnt_per_user_lag2,weekend_trip_cnt_per_user_roll4,weekday_match_rate_lag1,weekday_match_rate_lag2,weekday_match_rate_roll4,weekend_match_rate_lag1,weekend_match_rate_lag2,weekend_match_rate_roll4,coupon_BD_total_lag1,coupon_BD_total_lag2,coupon_BD_total_roll4,coupon_CDP_total_lag1,coupon_CDP_total_lag2,coupon_CDP_total_roll4,coupon_folk_total_lag1,coupon_folk_total_lag2,coupon_folk_total_roll4,coupon_growth_other_total_lag1,coupon_growth_other_total_lag2,coupon_growth_other_total_roll4,coupon_MGM_total_lag1,coupon_MGM_total_lag2,coupon_MGM_total_roll4,coupon_MKT_total_lag1,coupon_MKT_total_lag2,coupon_MKT_total_roll4,coupon_register_total_lag1,coupon_register_total_lag2,coupon_register_total_roll4,coupon_daily_total_lag1,coupon_daily_total_lag2,coupon_daily_total_roll4,delta_trip_per_user,delta_nonrepeat_per_user,ratio_trip_per_user,ratio_nonrepeat_per_user,face_value,face_value_num,coupon_BD_total_log1p,coupon_CDP_total_log1p,coupon_folk_total_log1p,coupon_growth_other_total_log1p,coupon_MGM_total_log1p,coupon_MKT_total_log1p,coupon_register_total_log1p,coupon_daily_total_log1p,coupon_BD_total_log1p_lag1,coupon_BD_total_log1p_lag2,coupon_BD_total_log1p_roll4,coupon_CDP_total_log1p_lag1,coupon_CDP_total_log1p_lag2,coupon_CDP_total_log1p_roll4,coupon_folk_total_log1p_lag1,coupon_folk_total_log1p_lag2,coupon_folk_total_log1p_roll4,coupon_growth_other_total_log1p_lag1,coupon_growth_other_total_log1p_lag2,coupon_growth_other_total_log1p_roll4,coupon_MGM_total_log1p_lag1,coupon_MGM_total_log1p_lag2,coupon_MGM_total_log1p_roll4,coupon_MKT_total_log1p_lag1,coupon_MKT_total_log1p_lag2,coupon_MKT_total_log1p_roll4,coupon_register_total_log1p_lag1,coupon_register_total_log1p_lag2,coupon_register_total_log1p_roll4,coupon_daily_total_log1p_lag1,coupon_daily_total_log1p_lag2,coupon_daily_total_log1p_roll4
0,2025-11-03,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,193,53,42,41,12,38,4,0.274611,0.217617,0.212435,0.062176,0.196891,0.020725,0.853684,0.752500,82,124,386,6,0,25,6,0,0.000000,0.000000,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,2,4.418841,4.828314,5.958425,1.945910,0.000000,3.258097,1.945910,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-11-10,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,71,15,11,12,3,6,5,0.211268,0.154930,0.169014,0.042254,0.084507,0.070423,0.916667,0.830000,24,29,142,0,0,0,6,0,0.244898,0.228571,0.0,0,0.274611,NaN,NaN,0.217617,NaN,NaN,0.212435,NaN,NaN,0.062176,NaN,NaN,0.196891,NaN,NaN,0.020725,NaN,NaN,0.853684,NaN,NaN,0.752500,NaN,NaN,82.0,NaN,NaN,124.0,NaN,NaN,386.0,NaN,NaN,6.0,NaN,NaN,0.0,NaN,NaN,25.0,NaN,NaN,6.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,15,2,3.218876,3.401197,4.962845,0.000000,0.000000,0.000000,1.945910,0.0,4.418841,NaN,NaN,4.828314,NaN,NaN,

In [38]:
import pandas as pd
import numpy as np
import re

# ----------------------------
# Drop experiment_date == 2026-01-12 then split and save
# ----------------------------
# 確保 experiment_date 是 datetime（若已是 datetime 也不影響）
df["experiment_date"] = pd.to_datetime(df["experiment_date"], errors="coerce")

# 拿掉 2026-01-12 的資料
df = df[df["experiment_date"] != pd.Timestamp("2026-01-12")].copy()

# ----------------------------
# Split and save
# ----------------------------
out_random = "../final_data/data_260125_random.csv"
out_control = "../final_data/data_260125_control.csv"

df_random = df[df["source"].astype(str).eq("隨機組")].copy()
df_control = df[df["source"].astype(str).eq("控制組")].copy()
if df_control.empty:
    df_control = df[df["source"].astype(str).str.contains("控制", na=False)].copy()

df.to_csv("../final_data/data_260125.csv", index=False, encoding="utf-8-sig")
df_random.to_csv(out_random, index=False, encoding="utf-8-sig")
df_control.to_csv(out_control, index=False, encoding="utf-8-sig")

print("rows total:", len(df))
print("rows random:", len(df_random))
print("rows control:", len(df_control))
print("Saved:", out_random, out_control)

rows total: 2856
rows random: 2304
rows control: 552
Saved: ../final_data/data_260125_random.csv ../final_data/data_260125_control.csv
